<a href="https://colab.research.google.com/github/damiancyrana/colab-notebooks/blob/main/ARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas statsmodels matplotlib pmdarima
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pmdarima as pm
import pickle
import os
from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA
from datetime import datetime
import pytz
import re
from statsmodels.tsa.statespace.sarimax import SARIMAX



In [4]:
# Ładowanie danych
url = '/content/drive/MyDrive/Colab Notebooks/US100_M5.csv'
data = pd.read_csv(url)

# Konwersja kolumny 'ctmString' na typ datetime i ustawienie jako indeks
data['ctmString'] = pd.to_datetime(data['ctmString'])
data.set_index('ctmString', inplace=True)

# Dodanie kolumny z dniem tygodnia
data['day_of_week'] = data.index.dayofweek

# Usunięcie weekendów (sobota = 5, niedziela = 6) i niepotrzebnych kolumn
data = data[~data['day_of_week'].isin([5, 6])]
data.drop(['Unnamed: 0', 'day_of_week'], axis=1, inplace=True)

# Sortowanie danych
data.sort_values('ctmString', inplace=True)
data.dropna(subset=['open', 'close', 'high', 'low', 'vol'], inplace=True)

**Funkcja do prognozowania z automatycznym doborem parametrów ARIMA**

In [ ]:
def auto_arima_forecast(feature_name):
    """
    Funkcja do prognozowania z automatycznym doborem parametrów ARIMA
    """
    auto_model = auto_arima(data[feature_name], seasonal=False, m=0,
                            stepwise=True, suppress_warnings=True,
                            error_action='ignore', trace=True,
                            max_order=None)

    # Użycie dobrych parametrów do stworzenia modelu ARIMA
    model = ARIMA(data[feature_name], order=auto_model.order)
    model_fit = model.fit()

    # Prognozowanie następnej wartości
    forecast = model_fit.forecast(steps=1)
    return forecast.iloc[0]

# Przeprowadzenie prognozowania dla każdej cechy
forecast_open = round(auto_arima_forecast('open'), 2)
forecast_close = round(auto_arima_forecast('close'), 2)
forecast_high = round(auto_arima_forecast('high'), 2)
forecast_low = round(auto_arima_forecast('low'), 2)

# Wyświetlenie zaokrąglonych prognoz
print("Prognozowane wartości dla kolejnej świecy M5 (zaokrąglone):")
print(f"Open: {forecast_open}")
print(f"Close: {forecast_close}")
print(f"High: {forecast_high}")
print(f"Low: {forecast_low}")


In [ ]:
def sarimax_forecast(feature_name):
    """
    Funkcja do prognozowania z automatycznym doborem parametrów SARIMAX
    """
    # Auto ARIMA do znalezienia najlepszego zestawu parametrów
    auto_model = auto_arima(data[feature_name], exogenous=data[['vol']],
                            seasonal=False, m=0,  # m to liczba okresów w sezonie
                            stepwise=True, suppress_warnings=True,
                            error_action='ignore', trace=True)

    # Budowanie modelu SARIMAX z najlepszymi parametrami
    model = SARIMAX(data[feature_name], exog=data['vol'],
                    order=auto_model.order, seasonal_order=auto_model.seasonal_order)
    model_fit = model.fit(disp=False)

    # Prognozowanie następnej wartości
    forecast = model_fit.forecast(steps=1, exog=data['vol'].iloc[-1])
    return forecast.iloc[0]

# Przeprowadzenie prognozowania dla każdej cechy
forecast_open = round(sarimax_forecast('open'), 2)
forecast_close = round(sarimax_forecast('close'), 2)
forecast_high = round(sarimax_forecast('high'), 2)
forecast_low = round(sarimax_forecast('low'), 2)

# Wyświetlenie zaokrąglonych prognoz
print("Prognozowane wartości dla kolejnej świecy M5 (zaokrąglone):")
print(f"Open: {forecast_open}")
print(f"Close: {forecast_close}")
print(f"High: {forecast_high}")
print(f"Low: {forecast_low}")

SARIMAX  Skuteczność modelu: 99.99655093973188 %

Open: 16018.22
Close: 16018.11
High: 16020.31
Low: 16017.02

In [5]:
def sarimax_forecast(feature_name):
    """
    Funkcja do prognozowania z zastosowaniem modelu SARIMAX z zadanymi parametrami.
    """
    # Zdefiniowane parametry ARIMA(2,1,1)
    order = (2, 1, 1)
    seasonal_order = (0, 0, 0, 0)  # Brak sezonowości

    # Budowanie modelu SARIMAX z zadanymi parametrami
    model = SARIMAX(data[feature_name], exog=data['vol'], order=order, seasonal_order=seasonal_order)
    model_fit = model.fit(disp=False)

    # Prognozowanie następnej wartości
    forecast = model_fit.forecast(steps=1, exog=data['vol'].iloc[-1])
    return forecast.iloc[0]

# Przeprowadzenie prognozowania dla każdej cechy
forecast_open = round(sarimax_forecast('open'), 2)
forecast_close = round(sarimax_forecast('close'), 2)
forecast_high = round(sarimax_forecast('high'), 2)
forecast_low = round(sarimax_forecast('low'), 2)

# Wyświetlenie zaokrąglonych prognoz
print("Prognozowane wartości dla kolejnej świecy M5 (zaokrąglone):")
print(f"Open: {forecast_open}")
print(f"Close: {forecast_close}")
print(f"High: {forecast_high}")
print(f"Low: {forecast_low}")


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/

Prognozowane wartości dla kolejnej świecy M5 (zaokrąglone):
Open: 16018.19
Close: 16017.92
High: 16021.59
Low: 16016.96


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [6]:
# Prognozowane wartości
predicted = {
    "Open": forecast_open,
    "Close": forecast_close,
    "High": forecast_high,
    "Low": forecast_low
}

# Rzeczywiste wartości (podane ręcznie)
actual = {
    "Open": 16017.37,
    "Close": 16017.87,
    "High": 16021.18,
    "Low": 16016.77
}

# Obliczenie błędu procentowego dla każdej cechy
percentage_errors = {}
for key in predicted.keys():
    error = abs(predicted[key] - actual[key]) / actual[key]
    percentage_errors[key] = error * 100

# Średni błąd procentowy
average_error = sum(percentage_errors.values()) / len(percentage_errors)
model_accuracy = 100 - average_error
print(f"Skuteczność modelu: {model_accuracy} %")


Skuteczność modelu: 99.9977057593497 %


In [ ]:
# Stworzenie DataFrame z jedną świecą
df_predicted = pd.DataFrame([predicted])

fig = go.Figure(data=[go.Candlestick(
                open=df_predicted['Open'],
                high=df_predicted['High'],
                low=df_predicted['Low'],
                close=df_predicted['Close'])])

fig.update_layout(
    title='Wykres Prognozowanej Świecy M5 US100',
    xaxis_title='Świeca M5',
    yaxis_title='Cena',
    xaxis_rangeslider_visible=False,
    yaxis=dict(
        tickformat=".2f"
    ),
    xaxis=dict(
        type='category'
    )
)

fig.show()
